# 2021-12-03b More complete parallelisation

Results below are for commit: ### (Rerunning the code below for other commits is not likely/guaranteed to give the same results.)

I have now implemented use in `sc_fitting.SCPaperParameterSearch` of the per-parameterisation parallelisation provided by `parameter_search.ParameterSearch`. Below are the results of some testing, where I run exactly the same model fit (`oVAaoVAloBEooBEvoAI` with the smaller grid) first serially (`parallel=False` in the `SCPaperParameterSearch` constructor) and then with parallelisation. The first attempt (`par_v1`) didn't work out - on closer inspection all except the very first parameterisation in the parallel search had the same metrics values as the very last parameterisation of the serial search. This was because I was sending the mutable objects holding the model parameterisations as arguments to the `multiprocessing.Pool.apply_async()` call, and updating these mutable objects between calls. This was based on my understanding that all of the relevant memory from the calling process would be serialised and sent on to the parallelised worker. That turns ot not to have been correct, and since generating the full list of `apply_async()` is much faster than testing a parameterisation, almost all of those calls ended up seeing just the final parameterisation. When instead creating local copies of the parameter objects to use for the `apply_async()` calls, I get back the same results as in the serial search (comare `par_v2` to `ser` below).

Also a note on speed: I am indeed seeing what seems to be a 4x speedup of this parallelisation, matching my 4 CPU cores rather than my 8 logical cores; based on for example the answers [here](https://stackoverflow.com/questions/10403201/8-logical-threads-at-4-cores-will-at-a-maximum-run-4-times-faster-in-parallel) this suggests that my code is more CPU-intensive than memory/disk intensive - which makes sense. 

In [1]:
%run ../SCPaper/do_2_analyse_deterministic_fits

['C:\\GITHUB\\COMMOTIONSFramework\\SCPaper/results\\DetFit_oVAaoVAloBEooBEvoAI_0_ser.pkl', 'C:\\GITHUB\\COMMOTIONSFramework\\SCPaper/results\\DetFit_oVAaoVAloBEooBEvoAI_1_par_v1.pkl', 'C:\\GITHUB\\COMMOTIONSFramework\\SCPaper/results\\DetFit_oVAaoVAloBEooBEvoAI_2_par_v2.pkl']

Loading parameter search from file "C:\GITHUB\COMMOTIONSFramework\SCPaper/results\DetFit_oVAaoVAloBEooBEvoAI_0_ser.pkl"...
	Done.
Analysing model oVAaoVAloBEooBEvoAI, 768 parameterisations...
	Main criteria:
		Vehicle asserting priority: Found 0 (0.0 %) parameterisations.
		Vehicle short-stopping: Found 36 (4.7 %) parameterisations.
		Pedestrian hesitation in deceleration scenario: Found 0 (0.0 %) parameterisations.
		Pedestrian starting before vehicle at full stop: Found 768 (100.0 %) parameterisations.
	Secondary criteria:
		Pedestrian hesitation in constant-speed scenario: Found 0 (0.0 %) parameterisations.
	All main criteria met: Found 0 (0.0 %) parameterisations.
	Max no of main criteria met was 2, for 36 pa